# Imports

In [2]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OrdinalEncoder
from sklearn.impute import KNNImputer
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

# Charger, observer et comprendre le dataset

# Transformer les données catégorielles en données numériques

In [4]:
dataset = pd.read_csv('credit.csv')
dataset.head()


,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,LP001002,Male,No,0,Graduate,No,5849,0.0,NaN,360.0,1.0,Urban,Y
1,LP001003,Male,Yes,1,Graduate,No,4583,1508.0,128.0,360.0,1.0,Rural,N
2,LP001005,Male,Yes,0,Graduate,Yes,3000,0.0,66.0,360.0,1.0,Urban,Y
3,LP001006,Male,Yes,0,Not Graduate,No,2583,2358.0,120.0,360.0,1.0,Urban,Y
4,LP001008,Male,No,0,Graduate,No,6000,0.0,141.0,360.0,1.0,Urban,Y


In [5]:

# 创建一个 OrdinalEncoder 对象，用于将分类特征转换为数字编码
enc = OrdinalEncoder()

# 从原始数据集中删除不需要作为特征的列：'Loan_ID' 和 'Loan_Status'
# 'Loan_ID' 是唯一标识符，不包含有用信息；'Loan_Status' 是目标变量 y
X = dataset.drop(['Loan_ID', 'Loan_Status'], axis=1)

# 定义需要进行编码的分类特征的列名
categorical_columes = ['Gender','Married','Dependents', 'Education', 'Self_Employed', 'Property_Area']

# 对这些分类列进行整数编码（Ordinal Encoding）
# 例如：['Male', 'Female'] => [1.0, 0.0] （具体值取决于fit_transform的顺序）
# 注意：OrdinalEncoder 默认按字母顺序将类别转换为从 0 开始的整数
X[categorical_columes] = enc.fit_transform(X[categorical_columes])

# 提取目标变量 'Loan_Status'，它表示是否批准贷款
y = dataset['Loan_Status']

# 查看前20行转换后的特征数据（X）
X.head(20)


,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area
0,1.0,0.0,0.0,0.0,0.0,5849,0.0,NaN,360.0,1.0,2.0
1,1.0,1.0,1.0,0.0,0.0,4583,1508.0,128.0,360.0,1.0,0.0
2,1.0,1.0,0.0,0.0,1.0,3000,0.0,66.0,360.0,1.0,2.0
3,1.0,1.0,0.0,1.0,0.0,2583,2358.0,120.0,360.0,1.0,2.0
4,1.0,0.0,0.0,0.0,0.0,6000,0.0,141.0,360.0,1.0,2.0
5,1.0,1.0,2.0,0.0,1.0,5417,4196.0,267.0,360.0,1.0,2.0
6,1.0,1.0,0.0,1.0,0.0,2333,1516.0,95.0,360.0,1.0,2.0
7,1.0,1.0,3.0,0.0,0.0,3036,2504.0,158.0,360.0,0.0,1.0
8,1.0,1.0,2.0,0.0,0.0,4006,1526.0,168.0,360.0,1.0,2.0
9,1.0,1.0,1.0,0.0,0.0,12841,10968.0,349.0,360.0,1.0,1.0


# Afficher et traiter les données manquantes
Pour traiter les données manquantes, regarder `SimpleImputer` ou `KNNImputer`

In [9]:
# 创建一个 KNNImputer 对象，用于填补缺失值
# n_neighbors=5 表示用每个样本最近的 5 个邻居的值来推测缺失值
imputer = KNNImputer(n_neighbors=5)

# 使用 KNNImputer 对特征数据 X 进行缺失值填充
# fit_transform 会计算每个样本最近的 5 个邻居，然后根据这些邻居的值来填补缺失的数据
# 返回的是一个 NumPy 数组（不再是 pandas DataFrame）
X = imputer.fit_transform(X)

X = pd.DataFrame(X, columns=dataset.drop(['Loan_ID', 'Loan_Status'], axis=1).columns)

# Standardiser les données

In [ ]:
scaler = StandardScaler()

# numerical_column = ['Dependents','ApplicantIncome','CoapplicantIncome','LoanAmount','Loan_Amount_Term','Property_Area']
# X[numerical_column] = scaler.fit_transform(X[numerical_column])

X = scaler.fit_transform(X)
X


array([[ 0.4821753 , -1.37110552, -0.76283716, ...,  0.27958831,
         0.44857574,  1.22329839],
       [ 0.4821753 ,  0.73143891,  0.23244974, ...,  0.27958831,
         0.44857574, -1.31851281],
       [ 0.4821753 ,  0.73143891, -0.76283716, ...,  0.27958831,
         0.44857574,  1.22329839],
       ...,
       [ 0.4821753 ,  0.73143891,  0.23244974, ...,  0.27958831,
         0.44857574,  1.22329839],
       [ 0.4821753 ,  0.73143891,  1.22773664, ...,  0.27958831,
         0.44857574,  1.22329839],
       [-2.11025581, -1.37110552, -0.76283716, ...,  0.27958831,
        -2.39086246, -0.04760721]])

# Séparer le dataset en train et test

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y)

# Régression logistique
Entrainer une régression logistique avec les hyper-paramètres par défaut

In [ ]:
clf = LogisticRegression()
clf.fit(X_train, y_train)


y_pred_test = clf.predict(X_test)
# for i in range(len(y_pred_test)):
#     print(str(list(y_pred_test)[i]) + ' vs ' + str(list(y_test)[i]))


In [ ]:
clf.score(X_test, y_test)

0.8051948051948052

In [ ]:
clf.score(X_train, y_train)

0.8130434782608695

# KNN
Entrainer un KNN avec les hyper-paramètres par défaut
Avec `GridSearchCV` ou `RandomizedSearchCV`, optimiser les hyper-paramètres. Utiliser une validation croisée de 5 splits. Vous explorerez les paramètres suivants:
- k : 3, 5, 7, 9, 11,
- poids : uniform, distance,
- distances : euclidean, manhattan, minkowski

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report, confusion_matrix

# 定义 KNN 模型
knn = KNeighborsClassifier()

# 定义超参数搜索空间
param_grid = {
    'n_neighbors': [3, 5, 7, 9, 11],  # k 值
    'weights': ['uniform', 'distance'],  # 权重
    'metric': ['euclidean', 'manhattan', 'minkowski']  # 距离度量
}

# 使用 5 折交叉验证进行网格搜索
grid_search = GridSearchCV(knn, param_grid, cv=5, scoring='accuracy', n_jobs=-1)
grid_search.fit(X_train, y_train)

# 输出最佳超参数
print("最佳超参数:", grid_search.best_params_)
print("最佳交叉验证准确率:", grid_search.best_score_)

# 使用最佳参数的模型进行预测
best_knn = grid_search.best_estimator_
y_pred = best_knn.predict(X_test)

# 评估模型性能
print("\n分类报告 (测试集):")
print(classification_report(y_test, y_pred))

# 显示混淆矩阵
print("\n混淆矩阵:")
print(confusion_matrix(y_test, y_pred))

最佳超参数: {'metric': 'euclidean', 'n_neighbors': 7, 'weights': 'uniform'}
最佳交叉验证准确率: 0.7956521739130434

分类报告 (测试集):
              precision    recall  f1-score   support

           N       0.88      0.47      0.61        49
           Y       0.80      0.97      0.88       105

    accuracy                           0.81       154
   macro avg       0.84      0.72      0.74       154
weighted avg       0.82      0.81      0.79       154


混淆矩阵:
[[ 23  26]
 [  3 102]]


# SVM
Entrainer un SVM avec les hyper-paramètres par défaut
Avec `GridSearchCV` ou `RandomizedSearchCV`, optimiser les hyper-paramètres. Utiliser une validation croisée de 5 splits. Vous explorerez les paramètres suivants:
- C : 0.01, 0.1, 1, 10, 100,
- noyau : linear, poly, rbf, sigmoid,
- gamma : scale, auto, 0.001, 0.01, 0.1, 1,
- degrée du polynome : 2, 3, 4, 5

In [ ]:
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report, confusion_matrix

# Définir le modèle SVM
svm = SVC()

# Définir la grille de recherche des hyperparamètres
param_grid = {
    'C': [0.01, 0.1, 1, 10, 100],          # Paramètre de régularisation
    'kernel': ['linear', 'poly', 'rbf', 'sigmoid'],  # Type de noyau
    'gamma': ['scale', 'auto', 0.001, 0.01, 0.1, 1],  # Paramètre gamma
    'degree': [2, 3, 4, 5]  # Degré du polynôme pour le noyau 'poly'
}

# Utilisation de GridSearchCV avec validation croisée à 5 splits
grid_search = GridSearchCV(svm, param_grid, cv=5, scoring='accuracy', n_jobs=-1)
grid_search.fit(X_train, y_train)

# Afficher les meilleurs paramètres
print("Meilleurs hyperparamètres:", grid_search.best_params_)
print("Meilleure précision en validation croisée:", grid_search.best_score_)

# Évaluer le modèle avec les meilleurs hyperparamètres
best_svm = grid_search.best_estimator_
y_pred = best_svm.predict(X_test)

# Afficher le rapport de classification
print("\nRapport de classification (test set) :")
print(classification_report(y_test, y_pred))

# Afficher la matrice de confusion
print("\nMatrice de confusion :")
print(confusion_matrix(y_test, y_pred))



Meilleurs hyperparamètres: {'C': 1, 'degree': 2, 'gamma': 0.01, 'kernel': 'rbf'}
Meilleure précision en validation croisée: 0.8108695652173914

Rapport de classification (test set) :
              precision    recall  f1-score   support

           N       0.92      0.45      0.60        49
           Y       0.79      0.98      0.88       105

    accuracy                           0.81       154
   macro avg       0.85      0.71      0.74       154
weighted avg       0.83      0.81      0.79       154


Matrice de confusion :
[[ 22  27]
 [  2 103]]


# DecisionTree
Entrainer un arbre de décision avec les hyper-paramètres par défaut
Avec `GridSearchCV` ou `RandomizedSearchCV`, optimiser les hyper-paramètres. Utiliser une validation croisée de 5 splits. Vous explorerez les hyper-paramètres suivants:
- critère : gini, entropy,
- profondeur maximale : None, 10, 20, 30, 40, 50,
- nombre minimum d'exemples par split : 2, 5, 10, 20,
- nombre minimum d'exemples par feuille : 1, 2, 4, 6,
- nombre de features maximum : None, sqrt, log2

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report, confusion_matrix

# Définir le modèle Decision Tree
tree = DecisionTreeClassifier()

# Définir la grille de recherche des hyperparamètres
param_grid = {
    'criterion': ['gini', 'entropy'],  # Fonction de mesure de qualité
    'max_depth': [None, 10, 20, 30, 40, 50],  # Profondeur max
    'min_samples_split': [2, 5, 10, 20],  # Min échantillons pour split
    'min_samples_leaf': [1, 2, 4, 6],  # Min échantillons par feuille
    'max_features': [None, 'sqrt', 'log2']  # Nombre max de features par split
}

# Utilisation de GridSearchCV avec validation croisée à 5 splits
grid_search = GridSearchCV(tree, param_grid, cv=5, scoring='accuracy', n_jobs=-1)
grid_search.fit(X_train, y_train)

# Afficher les meilleurs paramètres
print("Meilleurs hyperparamètres:", grid_search.best_params_)
print("Meilleure précision en validation croisée:", grid_search.best_score_)

# Évaluer le modèle avec les meilleurs hyperparamètres
best_tree = grid_search.best_estimator_
y_pred = best_tree.predict(X_test)

# Afficher le rapport de classification
print("\nRapport de classification (test set) :")
print(classification_report(y_test, y_pred))

# Afficher la matrice de confusion
print("\nMatrice de confusion :")
print(confusion_matrix(y_test, y_pred))


Meilleurs hyperparamètres: {'criterion': 'gini', 'max_depth': 40, 'max_features': 'sqrt', 'min_samples_leaf': 4, 'min_samples_split': 20}
Meilleure précision en validation croisée: 0.7804347826086957

Rapport de classification (test set) :
              precision    recall  f1-score   support

           N       0.63      0.49      0.55        49
           Y       0.78      0.87      0.82       105

    accuracy                           0.75       154
   macro avg       0.71      0.68      0.69       154
weighted avg       0.74      0.75      0.74       154


Matrice de confusion :
[[24 25]
 [14 91]]


# RandomForest
Entrainer une forêt aléatoire avec les paramètres par défaut
Avec `GridSearchCV` ou `RandomizedSearchCV`, optimiser les hyper-paramètres. Utiliser une validation croisée de 5 splits. Vous explorerez les hyper-paramètres suivants:
- nombre d'arbres : 50, 100, 200, 300,
- critère : gini, entropy,
- profondeur maximale : None, 10, 20, 30, 40, 50,
- nombre minimum d'exemples par split : 2, 5, 10, 20,
- nombre minimum d'exemples par feuille : 1, 2, 4, 6,
- nombre de features maximum : None, sqrt, log2,
- remise des exemples : vrai, faux

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier

# Définition du modèle Random Forest
rf = RandomForestClassifier(random_state=42)

# Définition de la grille de recherche
param_grid = {
    'n_estimators': [50, 100, 200, 300],
    'criterion': ['gini', 'entropy'],
    'max_depth': [None, 10, 20, 30, 40, 50],
    'min_samples_split': [2, 5, 10, 20],
    'min_samples_leaf': [1, 2, 4, 6],
    'max_features': [None, 'sqrt', 'log2'],
    'bootstrap': [True, False]
}

# Création du GridSearchCV avec 5-fold cross-validation
grid_search = GridSearchCV(rf, param_grid, cv=5, scoring='accuracy', n_jobs=-1)
grid_search.fit(X_train, y_train)

# Affichage des meilleurs hyperparamètres
print("Meilleurs hyperparamètres:", grid_search.best_params_)
print("Meilleure précision en validation croisée:", grid_search.best_score_)

# Utiliser le meilleur modèle trouvé
best_rf = grid_search.best_estimator_

# Prédictions sur le test set
y_pred = best_rf.predict(X_test)

# Rapport de classification
print("\nRapport de classification (test set) :")
print(classification_report(y_test, y_pred))

# Affichage de la matrice de confusion
print("\nMatrice de confusion :")
print(confusion_matrix(y_test, y_pred))

# ---------------------------------------------------------------------
# Création du RandomizedSearchCV
random_search = RandomizedSearchCV(rf, param_distributions=param_grid,
                                   n_iter=20, cv=5, scoring='accuracy',
                                   random_state=42, n_jobs=-1)

random_search.fit(X_train, y_train)

# Affichage des résultats
print("Meilleurs hyperparamètres:", random_search.best_params_)
print("Meilleure précision en validation croisée:", random_search.best_score_)


Meilleurs hyperparamètres: {'bootstrap': False, 'criterion': 'gini', 'max_depth': 10, 'max_features': 'sqrt', 'min_samples_leaf': 2, 'min_samples_split': 20, 'n_estimators': 100}
Meilleure précision en validation croisée: 0.8108695652173914

Rapport de classification (test set) :
              precision    recall  f1-score   support

           N       0.89      0.49      0.63        49
           Y       0.80      0.97      0.88       105

    accuracy                           0.82       154
   macro avg       0.85      0.73      0.76       154
weighted avg       0.83      0.82      0.80       154


Matrice de confusion :
[[ 24  25]
 [  3 102]]
Meilleurs hyperparamètres: {'n_estimators': 50, 'min_samples_split': 20, 'min_samples_leaf': 6, 'max_features': 'sqrt', 'max_depth': 10, 'criterion': 'entropy', 'bootstrap': False}
Meilleure précision en validation croisée: 0.8021739130434783


# Rapporter et analyser les résultats obtenir. Choisir le meilleur modèle

# Analyser le meilleur modèle
En utilisant `classification_report`, afficher différentes métriques en train et en test du modèle choisit. Afficher la matrice de confusion en train et en test de ce même modèle. Qu'en concluez-vous sur la qualité de ce modèle?

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

# Suppose que le meilleur modèle est trouvé après la comparaison des scores
best_model = best_rf  # Remplacez par le modèle choisi, ex: best_svm, best_knn, etc.

# Prédictions sur train et test set
y_train_pred = best_model.predict(X_train)
y_test_pred = best_model.predict(X_test)

# Afficher les scores de classification pour le train set
print("\nRapport de Classification (Train Set) :")
print(classification_report(y_train, y_train_pred))

# Afficher les scores de classification pour le test set
print("\nRapport de Classification (Test Set) :")
print(classification_report(y_test, y_test_pred))

# Affichage des matrices de confusion
print("\nMatrice de Confusion (Train Set) :")
print(confusion_matrix(y_train, y_train_pred))

print("\nMatrice de Confusion (Test Set) :")
print(confusion_matrix(y_test, y_test_pred))

# Afficher les scores d'accuracy
print("\nTrain Accuracy:", accuracy_score(y_train, y_train_pred))
print("Test Accuracy:", accuracy_score(y_test, y_test_pred))



Rapport de Classification (Train Set) :
              precision    recall  f1-score   support

           N       0.93      0.57      0.70       143
           Y       0.83      0.98      0.90       317

    accuracy                           0.85       460
   macro avg       0.88      0.77      0.80       460
weighted avg       0.86      0.85      0.84       460


Rapport de Classification (Test Set) :
              precision    recall  f1-score   support

           N       0.89      0.49      0.63        49
           Y       0.80      0.97      0.88       105

    accuracy                           0.82       154
   macro avg       0.85      0.73      0.76       154
weighted avg       0.83      0.82      0.80       154


Matrice de Confusion (Train Set) :
[[ 81  62]
 [  6 311]]

Matrice de Confusion (Test Set) :
[[ 24  25]
 [  3 102]]

Train Accuracy: 0.8521739130434782
Test Accuracy: 0.8181818181818182


# A faire à la maison : Développer un notebook similaire pour une tâche de régression